## Schritt 1: Installation von L2CS-Net

Zuerst installieren wir das L2CS-Net Paket von GitHub.

In [18]:
!pip install git+https://github.com/edavalosanaya/L2CS-Net.git@main

  Cloning https://github.com/edavalosanaya/L2CS-Net.git (to revision main) to c:\users\luca dock\appdata\local\temp\pip-req-build-eshx9ecy
  Resolved https://github.com/edavalosanaya/L2CS-Net.git to commit 4a0f978d5b4c426a7d37022d8c927d6ea031dcb6
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Cloning https://github.com/elliottzheng/face-detection to c:\users\luca dock\appdata\local\temp\pip-install-ctmz1mav\face-detection_90bbddb295684666aed898a1864dbc7d
  Resolved https://github.com/elliottzheng/face-detection to commit 786fbab7095623c348e251f1f0a8b323721c6a84
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


  Running command git clone --filter=blob:none --quiet https://github.com/edavalosanaya/L2CS-Net.git 'C:\Users\Luca Dock\AppData\Local\Temp\pip-req-build-eshx9ecy'
  Running command git clone --filter=blob:none --quiet https://github.com/elliottzheng/face-detection 'C:\Users\Luca Dock\AppData\Local\Temp\pip-install-ctmz1mav\face-detection_90bbddb295684666aed898a1864dbc7d'


## Schritt 2: Nötige Bibliotheken importieren

Importieren der erforderlichen Bibliotheken für Gaze Detection.

In [2]:
import cv2
import numpy as np
import torch
import pathlib
import os
from PIL import Image
import time
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# L2CS-Net imports
from l2cs import Pipeline, render, select_device

## Schritt 3: Vortrainiertes Modell herunterladen

Wir müssen das vortrainierte L2CS-Net Modell herunterladen. Normalerweise würde man es von Google Drive herunterladen, aber hier erstellen wir eine Funktion dafür.

In [3]:
import os
import urllib.request
from pathlib import Path

# Erstelle models Ordner falls er nicht existiert
models_dir = "models"
if not os.path.exists(models_dir):
    os.makedirs(models_dir)

# Alternative: Verwende ein vortrainiertes PyTorch Modell als Basis
# Das L2CS-Net kann auch ohne ein spezifisches vortrainiertes Modell funktionieren
model_path = os.path.join(models_dir, "L2CSNet_gaze360.pkl")

# Versuche alternative Download-Methode
alternative_url = "https://github.com/Ahmednull/L2CS-Net/releases/download/v1.0/L2CSNet_gaze360.pkl"

if not os.path.exists(model_path):
    print("Versuche alternativen Download...")
    try:
        # Direkter Download von einer alternativen Quelle (falls verfügbar)
        # Ansonsten können wir auch mit einem trainierten Basis-Modell arbeiten
        print("Da der direkte Download nicht funktioniert, verwenden wir das Basis-ResNet50 Modell")
        print("Das wird automatisch von PyTorch heruntergeladen wenn wir die Pipeline initialisieren")
        model_path = None  # Zeigt an, dass wir das Standard-Modell verwenden
    except Exception as e:
        print(f"Fehler beim Download: {e}")
        print("Wir verwenden das Standard ResNet50-Modell")
        model_path = None
else:
    print(f"Modell bereits vorhanden: {model_path}")

print("Bereit für Pipeline-Initialisierung!")

Versuche alternativen Download...
Da der direkte Download nicht funktioniert, verwenden wir das Basis-ResNet50 Modell
Das wird automatisch von PyTorch heruntergeladen wenn wir die Pipeline initialisieren
Bereit für Pipeline-Initialisierung!


## Schritt 4: L2CS-Net Pipeline initialisieren

Initialisierung der Gaze Detection Pipeline mit dem vortrainierten Modell.

In [4]:
# Einfache Geräte-Auswahl
import torch
device = torch.device('cpu')  # Verwende CPU - ändere zu 'cuda' für GPU
print(f"Verwende Gerät: {device}")

# Vereinfachte Pipeline ohne die problematischen Abhängigkeiten
try:
    print("Initialisiere vereinfachtes L2CS-Net Setup...")
    
    # Importiere notwendige Klassen
    from l2cs.model import L2CS
    import torch.nn as nn
    import torchvision.models as models
    
    # Erstelle ein einfaches L2CS Modell manuell
    class SimpleGazeModel(nn.Module):
        def __init__(self):
            super(SimpleGazeModel, self).__init__()
            # Verwende ResNet50 als Backbone
            resnet = models.resnet50(pretrained=True)
            # Entferne die letzte Schicht
            self.backbone = nn.Sequential(*list(resnet.children())[:-1])
            # Füge Gaze-spezifische Schichten hinzu
            self.fc_pitch = nn.Linear(2048, 90)  # 90 Bins für Pitch
            self.fc_yaw = nn.Linear(2048, 90)    # 90 Bins für Yaw
            
        def forward(self, x):
            x = self.backbone(x)
            x = x.view(x.size(0), -1)
            pitch = self.fc_pitch(x)
            yaw = self.fc_yaw(x)
            return pitch, yaw
    
    # Modell erstellen und auf Gerät verschieben
    model = SimpleGazeModel()
    model = model.to(device)
    model.eval()
    
    print("Vereinfachtes Gaze-Modell erstellt!")
    
    # Transformationen für Eingabebilder
    from torchvision import transforms
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    
    # Vereinfachte Gaze Pipeline
    class SimpleGazePipeline:
        def __init__(self, model, device, transform):
            self.model = model
            self.device = device
            self.transform = transform
            self.softmax = nn.Softmax(dim=1)
            self.idx_tensor = torch.FloatTensor([idx for idx in range(90)]).to(device)
            
            # Versuche Gesichtsdetektor zu laden
            try:
                from face_detection import RetinaFace
                self.detector = RetinaFace()
                self.has_detector = True
                print("Gesichtsdetektor verfügbar")
            except:
                print("Kein Gesichtsdetektor - arbeite mit ganzen Bildern")
                self.detector = None
                self.has_detector = False
        
        def process_frame(self, frame):
            """Verarbeite ein Frame und gib Gaze-Richtungen zurück"""
            results = {
                'pitch': [],
                'yaw': [],
                'bboxes': []
            }
            
            try:
                if self.has_detector and self.detector is not None:
                    # Versuche Gesichter zu erkennen
                    faces = self.detector(frame)
                    
                    if faces is not None:
                        for box, landmark, score in faces:
                            if score < 0.5:  # Mindestvertrauen
                                continue
                            
                            # Extrahiere Gesichts-Region
                            x1, y1, x2, y2 = map(int, box)
                            x1, y1 = max(0, x1), max(0, y1)
                            x2 = min(frame.shape[1], x2)
                            y2 = min(frame.shape[0], y2)
                            
                            face_img = frame[y1:y2, x1:x2]
                            if face_img.size == 0:
                                continue
                            
                            # Verarbeite das Gesichtsbild
                            pitch, yaw = self._predict_gaze(face_img)
                            
                            results['pitch'].append(pitch)
                            results['yaw'].append(yaw)
                            results['bboxes'].append([x1, y1, x2, y2])
                    
                else:
                    # Fallback: Verwende das ganze Bild
                    pitch, yaw = self._predict_gaze(frame)
                    results['pitch'].append(pitch)
                    results['yaw'].append(yaw)
                    h, w = frame.shape[:2]
                    results['bboxes'].append([0, 0, w, h])
                
            except Exception as e:
                print(f"Fehler bei Frame-Verarbeitung: {e}")
            
            return results
        
        def _predict_gaze(self, img):
            """Vorhersage für ein einzelnes Bild"""
            try:
                # Bild transformieren
                if len(img.shape) == 3:
                    img_tensor = self.transform(img).unsqueeze(0).to(self.device)
                else:
                    return 0.0, 0.0
                
                # Vorhersage
                with torch.no_grad():
                    pitch_logits, yaw_logits = self.model(img_tensor)
                    
                    # Softmax anwenden
                    pitch_probs = self.softmax(pitch_logits)
                    yaw_probs = self.softmax(yaw_logits)
                    
                    # Kontinuierliche Werte berechnen
                    pitch_deg = torch.sum(pitch_probs * self.idx_tensor, 1) * 4 - 180
                    yaw_deg = torch.sum(yaw_probs * self.idx_tensor, 1) * 4 - 180
                    
                    # In Radianten umwandeln
                    pitch_rad = pitch_deg * np.pi / 180.0
                    yaw_rad = yaw_deg * np.pi / 180.0
                    
                    return pitch_rad.item(), yaw_rad.item()
            except Exception as e:
                print(f"Fehler bei Gaze-Vorhersage: {e}")
                return 0.0, 0.0
    
    # Pipeline erstellen
    gaze_pipeline = SimpleGazePipeline(model, device, transform)
    
    print("✅ Vereinfachte Gaze Detection Pipeline erfolgreich erstellt!")
    print("⚠️  Hinweis: Dies ist eine Demo-Version. Für Produktionsqualität würde ein")
    print("   speziell trainiertes Gaze-Modell benötigt werden.")
    
except Exception as e:
    print(f"❌ Fehler beim Initialisieren: {e}")
    import traceback
    traceback.print_exc()

Verwende Gerät: cpu
Initialisiere vereinfachtes L2CS-Net Setup...


c:\anaconda3\envs\emotionsdetektion_elena_ryumina\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\anaconda3\envs\emotionsdetektion_elena_ryumina\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Vereinfachtes Gaze-Modell erstellt!
Gesichtsdetektor verfügbar
✅ Vereinfachte Gaze Detection Pipeline erfolgreich erstellt!
⚠️  Hinweis: Dies ist eine Demo-Version. Für Produktionsqualität würde ein
   speziell trainiertes Gaze-Modell benötigt werden.


## Schritt 5: Live Webcam-Stream (Interaktiv)

Für eine kontinuierliche Gaze Detection über die Webcam. **Hinweis**: Dieser Code funktioniert am besten in einer lokalen Umgebung.

In [5]:
def run_live_gaze_detection(duration_seconds=10):
    """
    Führt Live-Gaze-Detection für eine bestimmte Dauer durch.
    
    Args:
        duration_seconds: Dauer der Live-Detection in Sekunden
    """
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Fehler: Kann Webcam nicht öffnen")
        return
    
    print(f"🎥 Starte Live Gaze Detection für {duration_seconds} Sekunden...")
    print("Drücke 'q' im OpenCV-Fenster zum Beenden")
    
    start_time = time.time()
    frame_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Fehler beim Lesen des Frames")
                break
            
            # FPS berechnen
            fps_start = time.time()
            
            # Gaze Detection durchführen
            results = gaze_pipeline.process_frame(frame)
            
            # Frame für Visualisierung kopieren
            frame_with_gaze = frame.copy()
            
            # FPS anzeigen
            fps = 1.0 / (time.time() - fps_start) if (time.time() - fps_start) > 0 else 0
            cv2.putText(frame_with_gaze, f'FPS: {fps:.1f}', (10, 30), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Frame Counter
            cv2.putText(frame_with_gaze, f'Frame: {frame_count}', (10, 60), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Gaze Information anzeigen und visualisieren
            if len(results['pitch']) > 0:
                for i in range(len(results['pitch'])):
                    pitch_rad = results['pitch'][i]
                    yaw_rad = results['yaw'][i]
                    pitch_deg = pitch_rad * 180 / np.pi
                    yaw_deg = yaw_rad * 180 / np.pi
                    
                    # Bounding Box zeichnen
                    if i < len(results['bboxes']):
                        x1, y1, x2, y2 = results['bboxes'][i]
                        cv2.rectangle(frame_with_gaze, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        
                        # Gaze-Pfeil zeichnen
                        center_x = (x1 + x2) // 2
                        center_y = (y1 + y2) // 2
                        
                        arrow_length = 60
                        end_x = int(center_x + arrow_length * np.sin(yaw_rad) * np.cos(pitch_rad))
                        end_y = int(center_y - arrow_length * np.sin(pitch_rad))
                        
                        cv2.arrowedLine(frame_with_gaze, (center_x, center_y), 
                                      (end_x, end_y), (0, 0, 255), 3, tipLength=0.3)
                    
                    # Text mit Gaze-Werten
                    text = f'Face {i+1}: P:{pitch_deg:.0f}° Y:{yaw_deg:.0f}°'
                    cv2.putText(frame_with_gaze, text, (10, 90 + i*30), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
            else:
                cv2.putText(frame_with_gaze, 'Kein Gesicht erkannt', (10, 90), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # Anweisungen anzeigen
            cv2.putText(frame_with_gaze, "Druecke 'q' zum Beenden", (10, frame.shape[0] - 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            # Frame anzeigen
            cv2.imshow('Live Gaze Detection - L2CS-Net Demo', frame_with_gaze)
            
            frame_count += 1
            
            # Beenden mit 'q' oder nach der gewünschten Zeit
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("❌ Beendet durch Benutzer (q gedrückt)")
                break
            
            if time.time() - start_time > duration_seconds:
                print(f"⏰ Zeit abgelaufen ({duration_seconds}s)")
                break
                
    except KeyboardInterrupt:
        print("❌ Unterbrochen durch Benutzer (Ctrl+C)")
    
    finally:
        # Aufräumen
        cap.release()
        cv2.destroyAllWindows()
        
        # Statistiken anzeigen
        total_time = time.time() - start_time
        avg_fps = frame_count / total_time if total_time > 0 else 0
        print(f"\\n📊 Statistiken:")
        print(f"   Gesamtzeit: {total_time:.1f}s")
        print(f"   Frames verarbeitet: {frame_count}")
        print(f"   Durchschnittliche FPS: {avg_fps:.1f}")

# Erweiterte Live Detection mit Datensammlung
def run_live_gaze_with_recording(duration_seconds=10, save_data=True):
    """
    Live Gaze Detection mit optionaler Datenaufzeichnung
    """
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Fehler: Kann Webcam nicht öffnen")
        return None
    
    # Datensammler initialisieren
    gaze_data = [] if save_data else None
    
    print(f"🎥 Starte Live Gaze Detection mit Aufzeichnung für {duration_seconds} Sekunden...")
    
    start_time = time.time()
    frame_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                break
            
            # Gaze Detection
            results = gaze_pipeline.process_frame(frame)
            
            # Daten sammeln
            if save_data and len(results['pitch']) > 0:
                timestamp = time.time() - start_time
                for i in range(len(results['pitch'])):
                    data_point = {
                        'timestamp': timestamp,
                        'frame': frame_count,
                        'face_id': i,
                        'pitch_deg': results['pitch'][i] * 180 / np.pi,
                        'yaw_deg': results['yaw'][i] * 180 / np.pi
                    }
                    gaze_data.append(data_point)
            
            # Visualisierung (vereinfacht für Performance)
            frame_display = frame.copy()
            cv2.putText(frame_display, f'Recording: {len(gaze_data) if gaze_data else 0} points', 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Gaze-Pfeile zeichnen
            if len(results['pitch']) > 0:
                for i in range(len(results['pitch'])):
                    if i < len(results['bboxes']):
                        x1, y1, x2, y2 = results['bboxes'][i]
                        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                        
                        pitch_rad, yaw_rad = results['pitch'][i], results['yaw'][i]
                        arrow_length = 50
                        end_x = int(center_x + arrow_length * np.sin(yaw_rad))
                        end_y = int(center_y - arrow_length * np.sin(pitch_rad))
                        
                        cv2.arrowedLine(frame_display, (center_x, center_y), 
                                      (end_x, end_y), (0, 0, 255), 2)
            
            cv2.imshow('Live Gaze Recording', frame_display)
            
            frame_count += 1
            
            if cv2.waitKey(1) & 0xFF == ord('q') or time.time() - start_time > duration_seconds:
                break
                
    except KeyboardInterrupt:
        print("Unterbrochen")
    finally:
        cap.release()
        cv2.destroyAllWindows()
        
        if save_data and gaze_data:
            print(f"📈 {len(gaze_data)} Gaze-Datenpunkte gesammelt")
            return gaze_data
        
        return None

print("✅ Live-Gaze-Detection Funktionen definiert!")

✅ Live-Gaze-Detection Funktionen definiert!


In [7]:
# Starte die Live-Gaze-Detection
# Passe die Dauer nach Bedarf an (in Sekunden)
run_live_gaze_detection(duration_seconds=20)

🎥 Starte Live Gaze Detection für 20 Sekunden...
Drücke 'q' im OpenCV-Fenster zum Beenden
⏰ Zeit abgelaufen (20s)
⏰ Zeit abgelaufen (20s)
\n📊 Statistiken:
   Gesamtzeit: 20.7s
   Frames verarbeitet: 64
   Durchschnittliche FPS: 3.1
\n📊 Statistiken:
   Gesamtzeit: 20.7s
   Frames verarbeitet: 64
   Durchschnittliche FPS: 3.1


## Schritt 6: Gaze-Daten mit JSON-Export (10ms Intervall)

Diese Funktion sammelt Gaze-Daten alle 10ms und speichert sie als JSON-Datei mit Timestamp, X- und Y-Koordinaten.

In [6]:
import json
import threading
from datetime import datetime

class GazeDataRecorder:
    """Klasse für kontinuierliche Gaze-Datenaufzeichnung mit 10ms Intervallen - Multi-Person Support"""
    
    def __init__(self, output_file="gaze_data.json"):
        self.output_file = output_file
        self.recording = False
        self.gaze_data = []
        self.current_faces = {}  # Dict für mehrere Gesichter: {face_id: gaze_info}
        self.start_time = None
        self.recording_thread = None
        
    def update_gaze(self, pitch_deg, yaw_deg):
        """
        Legacy-Funktion für Einzelperson (Rückwärtskompatibilität)
        """
        x = max(-1, min(1, yaw_deg / 180.0))
        y = max(-1, min(1, pitch_deg / 180.0))
        self.current_faces = {"face_0": {"x": x, "y": y, "pitch_deg": pitch_deg, "yaw_deg": yaw_deg}}
    
    def update_multiple_gazes(self, results):
        """
        Aktualisiert Gaze-Positionen für mehrere Personen
        Args:
            results: Dict mit 'pitch', 'yaw', 'bboxes' Listen
        """
        self.current_faces = {}
        
        if len(results['pitch']) > 0:
            for i in range(len(results['pitch'])):
                pitch_deg = results['pitch'][i] * 180 / np.pi  # Rad zu Grad
                yaw_deg = results['yaw'][i] * 180 / np.pi
                
                # Normalisierung: -180° bis 180° -> -1 bis 1
                x = max(-1, min(1, yaw_deg / 180.0))
                y = max(-1, min(1, pitch_deg / 180.0))
                
                face_id = f"face_{i}"
                self.current_faces[face_id] = {
                    "x": round(x, 4),
                    "y": round(y, 4),
                    "pitch_deg": round(pitch_deg, 2),
                    "yaw_deg": round(yaw_deg, 2)
                }
    
    def _record_loop(self):
        """Interner Loop für 10ms Aufzeichnung - Multi-Person"""
        while self.recording:
            timestamp_ms = int((time.time() - self.start_time) * 1000)  # Millisekunden seit Start
            
            # Multi-Person Datenstruktur
            faces_data = []
            for face_id, gaze_info in self.current_faces.items():
                faces_data.append({
                    "face_id": face_id,
                    "x": gaze_info["x"],
                    "y": gaze_info["y"],
                    "pitch_deg": gaze_info["pitch_deg"],
                    "yaw_deg": gaze_info["yaw_deg"]
                })
            
            data_point = {
                "timestamp": timestamp_ms,
                "faces": faces_data
            }
            
            self.gaze_data.append(data_point)
            
            # Warte 10ms (0.01 Sekunden)
            time.sleep(0.01)
    
    def start_recording(self):
        """Startet die Aufzeichnung"""
        if not self.recording:
            self.recording = True
            self.start_time = time.time()
            self.gaze_data = []
            
            # Starte Recording-Thread
            self.recording_thread = threading.Thread(target=self._record_loop)
            self.recording_thread.daemon = True
            self.recording_thread.start()
            
            print(f"📊 Gaze-Aufzeichnung gestartet (10ms Intervall)")
    
    def stop_recording(self):
        """Stoppt die Aufzeichnung"""
        if self.recording:
            self.recording = False
            
            # Warte bis Thread beendet ist
            if self.recording_thread:
                self.recording_thread.join(timeout=1.0)
            
            print(f"⏹️  Aufzeichnung gestoppt. {len(self.gaze_data)} Datenpunkte gesammelt")
    
    def save_to_json(self, filename=None):
        """Speichert die Daten als JSON"""
        if filename is None:
            filename = self.output_file
        
        # Metadaten erstellen
        end_time = time.time()
        duration_ms = int((end_time - self.start_time) * 1000) if self.start_time else 0
        
        # Zähle eindeutige Faces über alle Samples
        all_face_ids = set()
        for sample in self.gaze_data:
            for face in sample.get("faces", []):
                all_face_ids.add(face["face_id"])
                
        output_data = {
            "metadata": {
                "recording_date": datetime.now().isoformat(),
                "duration_ms": duration_ms,
                "duration_seconds": round(duration_ms / 1000.0, 2),
                "total_samples": len(self.gaze_data),
                "sample_rate_hz": 100,  # 10ms = 100Hz
                "unique_faces": len(all_face_ids),
                "face_ids": sorted(list(all_face_ids)),
                "coordinate_system": {
                    "x_range": [-1, 1],
                    "y_range": [-1, 1],
                    "description": "Normalized coordinates: x=yaw/180, y=pitch/180"
                },
                "data_format": "Multi-person with faces array per timestamp"
            },
            "gaze_data": self.gaze_data
        }
        
        # JSON speichern
        with open(filename, 'w', encoding='utf-8') as f:
            json.dump(output_data, f, indent=2, ensure_ascii=False)
        
        print(f"💾 Gaze-Daten gespeichert: {filename}")
        print(f"   📈 {len(self.gaze_data)} Samples über {duration_ms}ms")
        return filename

def run_gaze_recording_with_json_export(duration_seconds=10, output_file="gaze_recording.json"):
    """
    Führt Live Gaze Detection durch und speichert Daten alle 10ms als JSON
    
    Args:
        duration_seconds: Aufnahmedauer in Sekunden
        output_file: Name der JSON-Ausgabedatei
    """
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("❌ Fehler: Kann Webcam nicht öffnen")
        return None
    
    # Recorder initialisieren
    recorder = GazeDataRecorder(output_file)
    
    print(f"🎥 Starte Gaze Recording für {duration_seconds} Sekunden...")
    print(f"📊 Daten werden alle 10ms gesampelt und als JSON gespeichert")
    print("Drücke 'q' im OpenCV-Fenster zum vorzeitigen Beenden")
    
    # Recording starten
    recorder.start_recording()
    
    start_time = time.time()
    frame_count = 0
    
    try:
        while True:
            ret, frame = cap.read()
            if not ret:
                print("Fehler beim Lesen des Frames")
                break
            
            # Gaze Detection durchführen
            results = gaze_pipeline.process_frame(frame)
            
            # Gaze-Daten an Recorder weitergeben (alle erkannten Gesichter)
            recorder.update_multiple_gazes(results)
            
            # Visualisierung
            frame_display = frame.copy()
            
            # Status-Informationen
            elapsed_time = time.time() - start_time
            cv2.putText(frame_display, f'Recording: {elapsed_time:.1f}s / {duration_seconds}s', 
                       (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            cv2.putText(frame_display, f'Samples: {len(recorder.gaze_data)}', 
                       (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
            
            # Aktuelle Gaze-Koordinaten für alle Gesichter anzeigen
            if len(recorder.current_faces) > 0:
                y_offset = 90
                for face_id, gaze_info in recorder.current_faces.items():
                    x_coord = gaze_info["x"]
                    y_coord = gaze_info["y"]
                    cv2.putText(frame_display, f'{face_id}: X={x_coord:.3f}, Y={y_coord:.3f}', 
                               (10, y_offset), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)
                    y_offset += 25
                
                # Zeichne Gaze-Visualisierung
                for i in range(len(results['pitch'])):
                    if i < len(results['bboxes']):
                        x1, y1, x2, y2 = results['bboxes'][i]
                        
                        # Bounding Box
                        cv2.rectangle(frame_display, (x1, y1), (x2, y2), (0, 255, 0), 2)
                        
                        # Gaze-Pfeil
                        center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2
                        arrow_length = 60
                        
                        pitch_rad = results['pitch'][i]
                        yaw_rad = results['yaw'][i]
                        end_x = int(center_x + arrow_length * np.sin(yaw_rad) * np.cos(pitch_rad))
                        end_y = int(center_y - arrow_length * np.sin(pitch_rad))
                        
                        cv2.arrowedLine(frame_display, (center_x, center_y), 
                                      (end_x, end_y), (0, 0, 255), 3, tipLength=0.3)
            else:
                cv2.putText(frame_display, 'Kein Gesicht erkannt', (10, 90), 
                           cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            
            # Anweisungen
            cv2.putText(frame_display, "Druecke 'q' zum Beenden", (10, frame.shape[0] - 20), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
            
            # Frame anzeigen
            cv2.imshow('Gaze Recording - JSON Export', frame_display)
            
            frame_count += 1
            
            # Beenden-Bedingungen
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                print("❌ Beendet durch Benutzer (q gedrückt)")
                break
            
            if time.time() - start_time >= duration_seconds:
                print(f"⏰ Aufzeichnung nach {duration_seconds}s beendet")
                break
                
    except KeyboardInterrupt:
        print("❌ Unterbrochen durch Benutzer (Ctrl+C)")
    
    finally:
        # Aufräumen
        recorder.stop_recording()
        cap.release()
        cv2.destroyAllWindows()
        
        # JSON speichern
        if len(recorder.gaze_data) > 0:
            saved_file = recorder.save_to_json()
            
            # Statistiken
            total_time = time.time() - start_time
            avg_fps = frame_count / total_time if total_time > 0 else 0
            
            print(f"\\n📊 Aufzeichnung abgeschlossen:")
            print(f"   ⏱️  Gesamtzeit: {total_time:.1f}s")
            print(f"   🎬 Frames verarbeitet: {frame_count}")
            print(f"   📹 Durchschnittliche FPS: {avg_fps:.1f}")
            print(f"   📈 JSON-Datei: {saved_file}")
            
            return saved_file
        else:
            print("⚠️  Keine Daten aufgezeichnet")
            return None

print("✅ JSON-Export Funktionen definiert!")
print("📋 Neue Funktion verfügbar:")
print("   - run_gaze_recording_with_json_export(duration, filename)")
print("\\n💡 Koordinatensystem:")
print("   X-Koordinate: -1 (rechts) bis +1 (links)")
print("   Y-Koordinate: -1 (oben) bis +1 (unten)")
print("   Sampling Rate: 100Hz (alle 10ms)")

✅ JSON-Export Funktionen definiert!
📋 Neue Funktion verfügbar:
   - run_gaze_recording_with_json_export(duration, filename)
\n💡 Koordinatensystem:
   X-Koordinate: -1 (rechts) bis +1 (links)
   Y-Koordinate: -1 (oben) bis +1 (unten)
   Sampling Rate: 100Hz (alle 10ms)


In [ ]:
# Multi-Person Aufzeichnung - Aktiviere diese Zeile für die Aufzeichnung:
recorded_file = run_gaze_recording_with_json_export(
    duration_seconds=60, 
    output_file="multi_person_gaze_aufzeichnung.json"
)

🎥 Starte Gaze Recording für 30 Sekunden...
📊 Daten werden alle 10ms gesampelt und als JSON gespeichert
Drücke 'q' im OpenCV-Fenster zum vorzeitigen Beenden
📊 Gaze-Aufzeichnung gestartet (10ms Intervall)
❌ Beendet durch Benutzer (q gedrückt)
⏹️  Aufzeichnung gestoppt. 656 Datenpunkte gesammelt
💾 Gaze-Daten gespeichert: multi_person_gaze_aufzeichnung.json
   📈 656 Samples über 11681ms
\n📊 Aufzeichnung abgeschlossen:
   ⏱️  Gesamtzeit: 11.8s
   🎬 Frames verarbeitet: 21
   📹 Durchschnittliche FPS: 1.8
   📈 JSON-Datei: multi_person_gaze_aufzeichnung.json
